In [36]:
##slide_window=0.1
import sounddevice as sd
import numpy as np
import scipy.signal
import python_speech_features
import tensorflow as tf
from PyQt5.QtCore import QTime
import librosa

# This gets called every 0.5 seconds
def sound(window,s,m):
    
    S = np.abs(librosa.stft(window)) #將整個window音訊做stft，並轉成絕對值
    
    if np.sum(S) > 3000: #判斷S的總和是否>3000，如果>3000，代表有講話
        # Compute features
        mfccs = python_speech_features.base.mfcc(window, #輸入訊號
                                            samplerate=8000, #取樣率
                                            winlen=0.256, #音框涵蓋時間
                                            winstep=0.050, #音框間距離
                                            numcep=num_mfcc, #返回係數的量
                                            nfilt=26, #過濾器數量
                                            nfft=2048,#FFT大小
                                            preemph=0.0,#不用預強化濾波器
                                            ceplifter=0,#ROBUST
                                            appendEnergy=True,#True的話，第0個倒頻譜係數被替代成總音框能量的對數
                                            winfunc=np.hanning)#hanning window
        mfccs = mfccs.transpose()
        
        # Make prediction from model
        in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
        #設定輸入張量
        interpreter.set_tensor(input_details[0]['index'], in_tensor)
        #進行預測
        interpreter.invoke()
        #取得輸出張量
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        val = output_data[0]#取得預測值
        val = val.tolist() #np.ndarray to list
        list_val_max = max(val) #取得最大值
        list_val_maxIndex = val.index(max(val)) #取得最大值的索引  
        
        if list_val_max > 0.3:
            if list_val_maxIndex == 3:
                data_ho.append(str(m) + "分" + str(s) + "秒，出現:" + str(words[list_val_maxIndex]) + "，預測值:" + str(list_val_max))
            data.append(str(m) + "分" + str(s) + "秒，出現:" + str(words[list_val_maxIndex]) + "，預測值:" + str(list_val_max))
         
#main
# Parameters
num_mfcc = 23 #回傳mfcc的量
model_path = './tflite/appendEnergyTT.tflite'
words = ['backgroundNoise', 'ㄏㄧㄡ', 'ㄟ', '吼', '啦', '嗯', '的一個', '的這個', '的那個', '著', '那', '那那個', '阿']#答案對應到的字詞
data = []
data_ho = []
start = 0 #一開始的索引值
end = 800 #一開始的索引值
s = 0 #秒
m = 0 #分
duration = 180 #讀音檔的總時間
sample_rate = 8000 #取樣率

#載入音檔
y, sr = librosa.load("./desktop/chen.wav",sr=sample_rate,duration=duration) 

# Sliding window
window = np.zeros(8000)#取樣音頻數據變數

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


while True:
    s = s + 0.1 #增加秒數
    if(s == 60): #60秒 轉成 1分
        s = 0
        m = m + 1

    #window[:4000] = window[4000:] #把音訊載入window
    #window[4000:] = y[start:end] #把音訊載入window


    
    window[0:800] = window[800:1600]
    window[800:1600] = window[1600:2400]
    window[1600:2400] = window[2400:3200]
    window[2400:3200] = window[3200:4000]
    window[3200:4000] = window[4000:4800]
    window[4000:4800] = window[4800:5600]
    window[4800:5600] = window[5600:6400]
    window[5600:6400] =  window[6400:7200] 
    window[6400:7200] = window[7200:]
    window[7200:] = y[start:end]
    
    sound(window,s,m) #呼叫sound()

    if(end == (8000 * duration)): #如果移動到最後，break
        break
    
    start = start + 800 #向後移動
    end = end + 800 #向後移動  
    
    
for i in range(len(data)):
        print(data[i])
    

0分1.6000000000000003秒，出現:吼，預測值:0.5126973390579224
0分1.7000000000000004秒，出現:吼，預測值:0.7247958779335022
0分1.8000000000000005秒，出現:吼，預測值:0.5704231262207031
0分1.9000000000000006秒，出現:嗯，預測值:0.8335582613945007
0分2.0000000000000004秒，出現:嗯，預測值:0.9023571610450745
0分2.1000000000000005秒，出現:嗯，預測值:0.9649389386177063
0分2.2000000000000006秒，出現:嗯，預測值:0.9836629629135132
0分2.3000000000000007秒，出現:嗯，預測值:0.9935737252235413
0分2.400000000000001秒，出現:嗯，預測值:0.9484816789627075
0分2.500000000000001秒，出現:的一個，預測值:0.96155846118927
0分2.600000000000001秒，出現:的一個，預測值:0.9967857599258423
0分2.700000000000001秒，出現:的一個，預測值:0.9990149736404419
0分2.800000000000001秒，出現:的一個，預測值:0.9869110584259033
0分2.9000000000000012秒，出現:的一個，預測值:0.9960744380950928
0分3.0000000000000013秒，出現:的一個，預測值:0.9717615246772766
0分3.1000000000000014秒，出現:的一個，預測值:0.9466769099235535
0分3.2000000000000015秒，出現:的一個，預測值:0.3712254464626312
0分3.3000000000000016秒，出現:嗯，預測值:0.4730560779571533
0分3.4000000000000017秒，出現:吼，預測值:0.4510437846183777
0分3.5000000000000018秒，出現:的一個，預測值:0.896172

In [37]:
#slide_window=0.2
import sounddevice as sd
import numpy as np8888ㄚ
import scipy.signal
import python_speech_features
import tensorflow as tf
from PyQt5.QtCore import QTime
import librosa

# This gets called every 0.5 seconds
def sound(window,s,m):
    
    S = np.abs(librosa.stft(window)) #將整個window音訊做stft，並轉成絕對值
    
    if np.sum(S) > 3000: #判斷S的總和是否>3000，如果>3000，代表有講話
        # Compute features
        mfccs = python_speech_features.base.mfcc(window, #輸入訊號
                                            samplerate=8000, #取樣率
                                            winlen=0.256, #音框涵蓋時間
                                            winstep=0.050, #音框間距離
                                            numcep=num_mfcc, #返回係數的量
                                            nfilt=26, #過濾器數量
                                            nfft=2048,#FFT大小
                                            preemph=0.0,#不用預強化濾波器
                                            ceplifter=0,#ROBUST
                                            appendEnergy=True,#True的話，第0個倒頻譜係數被替代成總音框能量的對數
                                            winfunc=np.hanning)#hanning window
        mfccs = mfccs.transpose()
        
        # Make prediction from model
        in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
        #設定輸入張量
        interpreter.set_tensor(input_details[0]['index'], in_tensor)
        #進行預測
        interpreter.invoke()
        #取得輸出張量
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        val = output_data[0]#取得預測值
        val = val.tolist() #np.ndarray to list
        list_val_max = max(val) #取得最大值
        list_val_maxIndex = val.index(max(val)) #取得最大值的索引  
        
        if list_val_max > 0.3:
            if list_val_maxIndex == 3:
                data_ho.append(str(m) + "分" + str(s) + "秒，出現:" + str(words[list_val_maxIndex]) + "，預測值:" + str(list_val_max))
            data.append(str(m) + "分" + str(s) + "秒，出現:" + str(words[list_val_maxIndex]) + "，預測值:" + str(list_val_max))
         
#main
# Parameters
num_mfcc = 23 #回傳mfcc的量
model_path = './tflite/appendEnergyTT.tflite'
words = ['backgroundNoise', 'ㄏㄧㄡ', 'ㄟ', '吼', '啦', '嗯', '的一個', '的這個', '的那個', '著', '那', '那那個', '阿']#答案對應到的字詞
data = []
data_ho = []
start = 0 #一開始的索引值
end = 1600 #一開始的索引值
s = 0 #秒
m = 0 #分
duration = 180 #讀音檔的總時間
sample_rate = 8000 #取樣率

#載入音檔
y, sr = librosa.load("./desktop/chen.wav",sr=sample_rate,duration=duration) 

# Sliding window
window = np.zeros(8000)#取樣音頻數據變數

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


while True:
    s = s + 0.2 #增加秒數
    if(s == 60): #60秒 轉成 1分
        s = 0
        m = m + 1

    #window[:4000] = window[4000:] #把音訊載入window
    #window[4000:] = y[start:end] #把音訊載入window

    
    window[0:1600] = window[1600:3200]
    window[1600:3200] = window[3200:4800]
    window[3200:4800] = window[4800:6400]
    window[4800:6400] = window[6400:]
    window[6400:] = y[start:end]
    
    sound(window,s,m) #呼叫sound()

    if(end == (8000 * duration)): #如果移動到最後，break
        break
    
    start = start + 1600 #向後移動
    end = end + 1600 #向後移動  
    
    
for i in range(len(data)):
        print(data[i])
    

0分1.5999999999999999秒，出現:吼，預測值:0.5126973390579224
0分1.7999999999999998秒，出現:吼，預測值:0.5704231262207031
0分1.9999999999999998秒，出現:嗯，預測值:0.9023571610450745
0分2.1999999999999997秒，出現:嗯，預測值:0.9836629629135132
0分2.4秒，出現:嗯，預測值:0.9484816789627075
0分2.6秒，出現:的一個，預測值:0.9967857599258423
0分2.8000000000000003秒，出現:的一個，預測值:0.9869110584259033
0分3.0000000000000004秒，出現:的一個，預測值:0.9717615246772766
0分3.2000000000000006秒，出現:的一個，預測值:0.3712254464626312
0分3.400000000000001秒，出現:吼，預測值:0.4510437846183777
0分3.600000000000001秒，出現:的一個，預測值:0.5579981803894043
0分3.800000000000001秒，出現:的一個，預測值:0.7302936911582947
0分4.000000000000001秒，出現:嗯，預測值:0.5787613987922668
0分4.200000000000001秒，出現:的那個，預測值:0.5282567143440247
0分4.400000000000001秒，出現:的那個，預測值:0.7386350631713867
0分4.600000000000001秒，出現:嗯，預測值:0.5890113115310669
0分4.800000000000002秒，出現:嗯，預測值:0.956076443195343
0分5.000000000000002秒，出現:嗯，預測值:0.7019282579421997
0分5.200000000000002秒，出現:吼，預測值:0.9084010720252991
0分5.400000000000002秒，出現:吼，預測值:0.6722729206085205
0分5.600000000000002秒，出現:吼，

In [32]:
#slide_window=0.25
import sounddevice as sd
import numpy as np
import scipy.signal
import python_speech_features
import tensorflow as tf
from PyQt5.QtCore import QTime
import librosa

# This gets called every 0.5 seconds
def sound(window,s,m):
    
    S = np.abs(librosa.stft(window)) #將整個window音訊做stft，並轉成絕對值
    
    if np.sum(S) > 3000: #判斷S的總和是否>3000，如果>3000，代表有講話
        # Compute features
        mfccs = python_speech_features.base.mfcc(window, #輸入訊號
                                            samplerate=8000, #取樣率
                                            winlen=0.256, #音框涵蓋時間
                                            winstep=0.050, #音框間距離
                                            numcep=num_mfcc, #返回係數的量
                                            nfilt=26, #過濾器數量
                                            nfft=2048,#FFT大小
                                            preemph=0.0,#不用預強化濾波器
                                            ceplifter=0,#ROBUST
                                            appendEnergy=True,#True的話，第0個倒頻譜係數被替代成總音框能量的對數
                                            winfunc=np.hanning)#hanning window
        mfccs = mfccs.transpose()
        
        # Make prediction from model
        in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
        #設定輸入張量
        interpreter.set_tensor(input_details[0]['index'], in_tensor)
        #進行預測
        interpreter.invoke()
        #取得輸出張量
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        val = output_data[0]#取得預測值
        val = val.tolist() #np.ndarray to list
        list_val_max = max(val) #取得最大值
        list_val_maxIndex = val.index(max(val)) #取得最大值的索引  
        
        if list_val_max > 0.3:
            if list_val_maxIndex == 3:
                data_ho.append(str(m) + "分" + str(s) + "秒，出現:" + str(words[list_val_maxIndex]) + "，預測值:" + str(list_val_max))
        data.append(str(m) + "分" + str(s) + "秒，出現:" + str(words[list_val_maxIndex]) + "，預測值:" + str(list_val_max))
             
         
#main
# Parameters
num_mfcc = 23 #回傳mfcc的量
model_path = './tflite/appendEnergyTT.tflite'
words = ['backgroundNoise', 'ㄏㄧㄡ', 'ㄟ', '吼', '啦', '嗯', '的一個', '的這個', '的那個', '著', '那', '那那個', '阿']#答案對應到的字詞
data = []
data_ho = []
start = 0 #一開始的索引值
end = 2000 #一開始的索引值
s = 0 #秒
m = 0 #分
duration = 180 #讀音檔的總時間
sample_rate = 8000 #取樣率

#載入音檔
y, sr = librosa.load("./desktop/chen.wav",sr=sample_rate,duration=duration) 

# Sliding window
window = np.zeros(8000)#取樣音頻數據變數

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


while True:
    s = s + 0.25 #增加秒數
    if(s == 60): #60秒 轉成 1分
        s = 0
        m = m + 1

    #window[:4000] = window[4000:] #把音訊載入window
    #window[4000:] = y[start:end] #把音訊載入window
    window[0:2000] = window[2000:4000]
    window[2000:4000] = window[4000:6000]
    window[4000:6000] = window[6000:8000]
    window[6000:8000] = y[start:end]
    sound(window,s,m) #呼叫sound()

    if(end == (8000 * duration)): #如果移動到最後，break
        break
    
    start = start + 2000 #向後移動
    end = end + 2000 #向後移動    
    
    
for i in range(len(data)):
        print(data[i])

0分1.75秒，出現:吼，預測值:0.7691010236740112
0分2.0秒，出現:嗯，預測值:0.9023571610450745
0分2.25秒，出現:嗯，預測值:0.9866108894348145
0分2.5秒，出現:的一個，預測值:0.96155846118927
0分2.75秒，出現:的一個，預測值:0.9933411478996277
0分3.0秒，出現:的一個，預測值:0.9717615246772766
0分3.25秒，出現:嗯，預測值:0.44183462858200073
0分3.5秒，出現:的一個，預測值:0.896172821521759
0分3.75秒，出現:的一個，預測值:0.7100469470024109
0分4.0秒，出現:嗯，預測值:0.5787613987922668
0分4.25秒，出現:的那個，預測值:0.613018274307251
0分4.5秒，出現:的那個，預測值:0.273445725440979
0分4.75秒，出現:嗯，預測值:0.910406231880188
0分5.0秒，出現:嗯，預測值:0.7019282579421997
0分5.25秒，出現:吼，預測值:0.8297287821769714
0分5.5秒，出現:吼，預測值:0.8657661080360413
0分5.75秒，出現:那，預測值:0.951887845993042
0分6.0秒，出現:嗯，預測值:0.38835015892982483
0分6.25秒，出現:嗯，預測值:0.9837387800216675
0分6.5秒，出現:嗯，預測值:0.7983411550521851
0分6.75秒，出現:的一個，預測值:0.36101531982421875
0分7.0秒，出現:的一個，預測值:0.3445643186569214
0分7.25秒，出現:的那個，預測值:0.6504129767417908
0分7.5秒，出現:吼，預測值:0.8423324823379517
0分7.75秒，出現:的一個，預測值:0.6714931726455688
0分8.0秒，出現:的一個，預測值:0.982721745967865
0分8.25秒，出現:的一個，預測值:0.8018931150436401
0分8.5秒，出現:的一個，預測值:0.